# Data Acquisition Exercises

The end result of this exercise should be a file named acquire.py.

1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

2. Do the same thing, but for stores.

3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

4. Save the data in your files to local csv files so that it will be faster to access in the future.

5. Combine the data from your three separate dataframes into one large dataframe.

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [1]:
import requests
import pandas as pd

In [2]:
# query example from curriculum
url = 'https://swapi.dev/api/people/5'
response = requests.get(url)
print(response.text)

{"name":"Leia Organa","height":"150","mass":"49","hair_color":"brown","skin_color":"light","eye_color":"brown","birth_year":"19BBY","gender":"female","homeworld":"http://swapi.dev/api/planets/2/","films":["http://swapi.dev/api/films/1/","http://swapi.dev/api/films/2/","http://swapi.dev/api/films/3/","http://swapi.dev/api/films/6/"],"species":[],"vehicles":["http://swapi.dev/api/vehicles/30/"],"starships":[],"created":"2014-12-10T15:20:09.791000Z","edited":"2014-12-20T21:17:50.315000Z","url":"http://swapi.dev/api/people/5/"}


In [3]:
# convert the response to an object we can work with
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Leia Organa',
 'height': '150',
 'mass': '49',
 'hair_color': 'brown',
 'skin_color': 'light',
 'eye_color': 'brown',
 'birth_year': '19BBY',
 'gender': 'female',
 'homeworld': 'http://swapi.dev/api/planets/2/',
 'films': ['http://swapi.dev/api/films/1/',
  'http://swapi.dev/api/films/2/',
  'http://swapi.dev/api/films/3/',
  'http://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['http://swapi.dev/api/vehicles/30/'],
 'starships': [],
 'created': '2014-12-10T15:20:09.791000Z',
 'edited': '2014-12-20T21:17:50.315000Z',
 'url': 'http://swapi.dev/api/people/5/'}

In [4]:
# 1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data
# for items.
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



In [5]:
# the response says there is documentation
# request the documentation
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [6]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [7]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [8]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 3
next_page: /api/v1/items?page=2


In [9]:
data['payload']['items'][:2]

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

In [10]:
df = pd.DataFrame(data['payload']['items'])
df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [11]:
df.shape

(20, 6)

In [12]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: /api/v1/items?page=3


In [13]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: None


In [14]:
df.shape

(50, 8)

In [15]:
# 2. Do the same thing, but for stores

response = requests.get('https://python.zach.lol/api/v1/stores')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [16]:
data['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [17]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 1
next_page: None


In [18]:
data['payload']['stores']

[{'store_address': '12125 Alamo Ranch Pkwy',
  'store_city': 'San Antonio',
  'store_id': 1,
  'store_state': 'TX',
  'store_zipcode': '78253'},
 {'store_address': '9255 FM 471 West',
  'store_city': 'San Antonio',
  'store_id': 2,
  'store_state': 'TX',
  'store_zipcode': '78251'},
 {'store_address': '2118 Fredericksburg Rdj',
  'store_city': 'San Antonio',
  'store_id': 3,
  'store_state': 'TX',
  'store_zipcode': '78201'},
 {'store_address': '516 S Flores St',
  'store_city': 'San Antonio',
  'store_id': 4,
  'store_state': 'TX',
  'store_zipcode': '78204'},
 {'store_address': '1520 Austin Hwy',
  'store_city': 'San Antonio',
  'store_id': 5,
  'store_state': 'TX',
  'store_zipcode': '78218'},
 {'store_address': '1015 S WW White Rd',
  'store_city': 'San Antonio',
  'store_id': 6,
  'store_state': 'TX',
  'store_zipcode': '78220'},
 {'store_address': '12018 Perrin Beitel Rd',
  'store_city': 'San Antonio',
  'store_id': 7,
  'store_state': 'TX',
  'store_zipcode': '78217'},
 {'store

In [19]:
storesdf = pd.DataFrame(data['payload']['stores'])
storesdf.shape

(10, 5)

3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

In [21]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 183
next_page: /api/v1/sales?page=2


In [22]:
data['payload']['sales'][:2]

[{'item': 1,
  'sale_amount': 13.0,
  'sale_date': 'Tue, 01 Jan 2013 00:00:00 GMT',
  'sale_id': 1,
  'store': 1},
 {'item': 1,
  'sale_amount': 11.0,
  'sale_date': 'Wed, 02 Jan 2013 00:00:00 GMT',
  'sale_id': 2,
  'store': 1}]

In [23]:
sldf = pd.DataFrame(data['payload']['sales'])
slsdf.shape

(5000, 5)

In [25]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

salesdf = pd.concat([salesdf, pd.DataFrame(data['payload']['sales'])]).reset_index()

max_page: 183
next_page: /api/v1/sales?page=4


In [77]:
base_url2 = 'https://python.zach.lol'

def get_sales(base_url, df_name):
    # get first page and make dataframe
    response = requests.get('https://python.zach.lol/api/v1/sales')
    data = response.json()
    data.keys()
    print('max_page: %s' % data['payload']['max_page'])
    print('next_page: %s' % data['payload']['next_page'])
    df_name = pd.DataFrame(data['payload']['sales'])
    while data['payload']['next_page'] != 'None':
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        print('max_page: %s' % data['payload']['max_page'])
        print('next_page: %s' % data['payload']['next_page'])
        df_name = pd.concat([df_name, pd.DataFrame(data['payload']['sales'])])
        if data['payload']['next_page'] == None:
            break
    df_name = df_name.reset_index()
    print('full_shape', df_name.shape)
    return df_name

In [78]:
salesdf = get_sales(base_url2, salesdf)

max_page: 183
next_page: /api/v1/sales?page=2
max_page: 183
next_page: /api/v1/sales?page=3
max_page: 183
next_page: /api/v1/sales?page=4
max_page: 183
next_page: /api/v1/sales?page=5
max_page: 183
next_page: /api/v1/sales?page=6
max_page: 183
next_page: /api/v1/sales?page=7
max_page: 183
next_page: /api/v1/sales?page=8
max_page: 183
next_page: /api/v1/sales?page=9
max_page: 183
next_page: /api/v1/sales?page=10
max_page: 183
next_page: /api/v1/sales?page=11
max_page: 183
next_page: /api/v1/sales?page=12
max_page: 183
next_page: /api/v1/sales?page=13
max_page: 183
next_page: /api/v1/sales?page=14
max_page: 183
next_page: /api/v1/sales?page=15
max_page: 183
next_page: /api/v1/sales?page=16
max_page: 183
next_page: /api/v1/sales?page=17
max_page: 183
next_page: /api/v1/sales?page=18
max_page: 183
next_page: /api/v1/sales?page=19
max_page: 183
next_page: /api/v1/sales?page=20
max_page: 183
next_page: /api/v1/sales?page=21
max_page: 183
next_page: /api/v1/sales?page=22
max_page: 183
next_pa

max_page: 183
next_page: /api/v1/sales?page=175
max_page: 183
next_page: /api/v1/sales?page=176
max_page: 183
next_page: /api/v1/sales?page=177
max_page: 183
next_page: /api/v1/sales?page=178
max_page: 183
next_page: /api/v1/sales?page=179
max_page: 183
next_page: /api/v1/sales?page=180
max_page: 183
next_page: /api/v1/sales?page=181
max_page: 183
next_page: /api/v1/sales?page=182
max_page: 183
next_page: /api/v1/sales?page=183
max_page: 183
next_page: None
full_shape (913000, 6)


In [90]:
# 4. Save the data in your files to local csv files so that it will be faster to access in the future.
# employees.to_csv('employees_df.csv')

df.to_csv('items.csv')
storesdf.to_csv('stores.csv')
salesdf.to_csv('sales.csv')


In [69]:
df.head()

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [70]:
storesdf.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [84]:
salesdf.head()

,index,item,sale_amount,sale_date,sale_id,store
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [85]:
# 5.Combine the data from your three separate dataframes into one large dataframe.

fulldf = salesdf.merge(storesdf, left_on='store', right_on='store_id')

In [87]:
fulldf = fulldf.merge(df, left_on='item', right_on='item_id')

In [89]:
fulldf.shape

(913000, 19)

In [91]:
fulldf.to_csv('alldf.csv')

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [83]:
germany = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
germany.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [92]:
germany.isna().sum()

Date              0
Consumption       0
Wind           1463
Solar          2195
Wind+Solar     2196
dtype: int64

In [ ]:
def read_url_csv(url):
    df = pd.read_csv(url)
    return df